<a href="https://colab.research.google.com/github/Yepji/pm2.5/blob/main/%EA%B3%B5%EB%AA%A8%EC%A0%84_%EC%B5%9C%EC%A2%85(%EA%B0%84%EB%8B%A8%ED%95%98%EA%B2%8C_%EA%B5%90%ED%98%B8).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import numpy as np
import pandas as pd
import math
from sklearn.impute import KNNImputer

import torch
import torch.nn as nn
import torch.optim as optim

import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
from keras.preprocessing.sequence import TimeseriesGenerator

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error

import os
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


AWS - PM2.5 지역 매칭

In [ ]:
df1=pd.read_csv("/content/drive/MyDrive/TNT공모전/dataset/META/awsmap.csv")
df2=pd.read_csv("/content/drive/MyDrive/TNT공모전/dataset/META/pmmap.csv")

In [ ]:
aws_la=df1["Latitude"]
aws_lo=df1["Longitude"]
pm_la=df2["Latitude"]
pm_lo=df2["Longitude"]

In [ ]:
x = aws_la
y = aws_lo

for i in range(len(pm_la)):
    x0 = pm_la[i].item()
    y0 = pm_lo[i].item()
    distances = []
    for j in range(len(aws_la)):
        dist = math.hypot(x0 - x[j].item(), y0 - y[j].item())
        distances.append(dist)
    min_distance = min(distances)
    min_distance_index = distances.index(min_distance)
    print(f"가장 가까운 지점{x0, y0}까지의 최소 거리: {min_distance}")
    print(f"가장 가까운 지점의 인덱스: {min_distance_index}")

가장 가까운 지점(36.5122519, 127.2467891)까지의 최소 거리: 0.02021866247851108
가장 가까운 지점의 인덱스: 3
가장 가까운 지점(36.5928874, 127.2925497)까지의 최소 거리: 0.028784983044114722
가장 가까운 지점의 인덱스: 5
가장 가까운 지점(36.3682419, 127.3184975)까지의 최소 거리: 0.09529985184595152
가장 가까운 지점의 인덱스: 24
가장 가까운 지점(36.3172147, 127.4378249)까지의 최소 거리: 0.049287614226090244
가장 가까운 지점의 인덱스: 0
가장 가까운 지점(36.3723875, 127.4177136)까지의 최소 거리: 0.045933976980113325
가장 가까운 지점의 인덱스: 2
가장 가까운 지점(36.3044424, 127.3667423)까지의 최소 거리: 0.03198271638010016
가장 가까운 지점의 인덱스: 0
가장 가까운 지점(36.4469512, 127.119209)까지의 최소 거리: 0.039800943989304574
가장 가까운 지점의 인덱스: 12
가장 가까운 지점(36.1992169, 127.0870207)까지의 최소 거리: 0.02453373013016669
가장 가까운 지점의 인덱스: 14
가장 가까운 지점(36.3531477, 126.5897346)까지의 최소 거리: 0.09222935201144175
가장 가까운 지점의 인덱스: 26
가장 가까운 지점(36.9875793, 126.3916719)까지의 최소 거리: 0.023296160587099286
가장 가까운 지점의 인덱스: 9
가장 가까운 지점(36.7801577, 126.4551973)까지의 최소 거리: 0.16026739673613125
가장 가까운 지점의 인덱스: 18
가장 가까운 지점(36.7826998, 127.0146102)까지의 최소 거리: 0.1620040710108333
가장 가까운 지점의 인덱스

In [ ]:
mapped = {'아름동': '세종고운', '신흥동': '세종연서', '노은동': '계룡', '문창동': '오월드', 
          '읍내동': '세천', '정림동': '오월드', '공주': '공주', '논산': '논산', '대천2동': '대천항', 
          '독곶리': '대산', '동문동': '태안', '모종동': '아산', '신방동': '세종전의', '예산군': '예산', 
          '이원면': '태안', '홍성읍': '홍북', '성성동': '성거'}

for key, value in mapped.items():
    print(f"{key}의 PM2.5 값은 {value}의 기상데이터를 사용하여 예측했습니다.")

아름동의 PM2.5 값은 세종고운의 기상데이터를 사용하여 예측했습니다.
신흥동의 PM2.5 값은 세종연서의 기상데이터를 사용하여 예측했습니다.
노은동의 PM2.5 값은 계룡의 기상데이터를 사용하여 예측했습니다.
문창동의 PM2.5 값은 오월드의 기상데이터를 사용하여 예측했습니다.
읍내동의 PM2.5 값은 세천의 기상데이터를 사용하여 예측했습니다.
정림동의 PM2.5 값은 오월드의 기상데이터를 사용하여 예측했습니다.
공주의 PM2.5 값은 공주의 기상데이터를 사용하여 예측했습니다.
논산의 PM2.5 값은 논산의 기상데이터를 사용하여 예측했습니다.
대천2동의 PM2.5 값은 대천항의 기상데이터를 사용하여 예측했습니다.
독곶리의 PM2.5 값은 대산의 기상데이터를 사용하여 예측했습니다.
동문동의 PM2.5 값은 태안의 기상데이터를 사용하여 예측했습니다.
모종동의 PM2.5 값은 아산의 기상데이터를 사용하여 예측했습니다.
신방동의 PM2.5 값은 세종전의의 기상데이터를 사용하여 예측했습니다.
예산군의 PM2.5 값은 예산의 기상데이터를 사용하여 예측했습니다.
이원면의 PM2.5 값은 태안의 기상데이터를 사용하여 예측했습니다.
홍성읍의 PM2.5 값은 홍북의 기상데이터를 사용하여 예측했습니다.
성성동의 PM2.5 값은 성거의 기상데이터를 사용하여 예측했습니다.


TRAIN 파일의 결측치 처리

In [ ]:
def fill_missing_aws(df): #TRAIN_AWS 파일 결측치 처리하는 함수 정의
    df_loc = df.iloc[:, 3:] #연도, 일시, 지점을 제외한 부분
  
    imputer = KNNImputer(n_neighbors = 50)  
    df_fillna = imputer.fit_transform(df_loc)
    df_fillna = pd.DataFrame(df_fillna, columns=df_loc.columns)
    return df_fillna #결측치 처리

input_folder = "/content/drive/MyDrive/TNT공모전/dataset/TRAIN_AWS"  #폴더 경로를 입력

# 파일명-변수명 매핑 딕셔너리
file_aws_variable_dic = {"계룡.csv": "gyeryong_aws", "공주.csv": "gongju_aws", "논산.csv": "nonsan_aws", "당진.csv": "dangjin_aws", "대산.csv": "daesan_aws", 
                      "대천항.csv": "daecheonh_aws", "북격렬비도.csv": "northkyeok_aws", "서천.csv": "seocheon_aws", "성거.csv": "seonggeo_aws", 
                      "세종고운.csv": "sjgoun_aws", "세종금남.csv": "sjgeumnam_aws", "세종연서.csv": "sjyeonseo_aws", "세종전의.csv": "sjjeonui_aws", 
                      "세천.csv": "secheon_aws", "아산.csv": "asan_aws", "안도.csv": "ando_aws", "양화.csv": "yanghwa_aws", "예산.csv": "yesan_aws", 
                      "오월드.csv":"oworld_aws", "옹도.csv": "ongdo_aws", "유구.csv": "yougu_aws", "장동.csv": "jangdong_aws", "정산.csv": "jeongsan_aws", 
                      "정안.csv": "jeongan_aws", "청양.csv": "cheongyang_aws", "춘장대.csv": "chunjang_aws", "태안.csv": "taean_aws", "호도.csv": "hodo_aws", 
                      "홍북.csv": "hongbuk_aws", "홍성죽도.csv": "hongseongjukdo_aws"}

csv_files = [f for f in os.listdir(input_folder) if f.endswith('.csv')] 

# 각 CSV 파일에 대해 결측치 처리 및 동적 변수에 저장
for file in csv_files:
    if file in file_aws_variable_dic:
        file_path = os.path.join(input_folder, file)
        variable_name = file_aws_variable_dic[file]

        df = pd.read_csv(file_path)
        df_filled = fill_missing_aws(df)
        df_first = df.iloc[:, :3] #연도, 일시, 지점 부분
        result_df = pd.concat([df_first, df_filled], axis=1) #연도, 일시, 지점 부분과 뒷 칼럼 부분을 다시 연결


        # 처리된 데이터프레임을 동적으로 생성된 변수에 저장
        globals()[variable_name] = result_df

        print(f"{file} 결측치 처리 완료, 저장된 변수명: {variable_name}")
    else:
        print(f"{file}에 대한 변수명 매핑이 없습니다. 딕셔너리에 추가해 주세요.")

서천.csv 결측치 처리 완료, 저장된 변수명: seocheon_aws
북격렬비도.csv 결측치 처리 완료, 저장된 변수명: northkyeok_aws
대산.csv 결측치 처리 완료, 저장된 변수명: daesan_aws
계룡.csv 결측치 처리 완료, 저장된 변수명: gyeryong_aws
공주.csv 결측치 처리 완료, 저장된 변수명: gongju_aws
성거.csv 결측치 처리 완료, 저장된 변수명: seonggeo_aws
대천항.csv 결측치 처리 완료, 저장된 변수명: daecheonh_aws
세종고운.csv 결측치 처리 완료, 저장된 변수명: sjgoun_aws
논산.csv 결측치 처리 완료, 저장된 변수명: nonsan_aws
당진.csv 결측치 처리 완료, 저장된 변수명: dangjin_aws
옹도.csv 결측치 처리 완료, 저장된 변수명: ongdo_aws
정산.csv 결측치 처리 완료, 저장된 변수명: jeongsan_aws
장동.csv 결측치 처리 완료, 저장된 변수명: jangdong_aws
정안.csv 결측치 처리 완료, 저장된 변수명: jeongan_aws
양화.csv 결측치 처리 완료, 저장된 변수명: yanghwa_aws
예산.csv 결측치 처리 완료, 저장된 변수명: yesan_aws
세종전의.csv 결측치 처리 완료, 저장된 변수명: sjjeonui_aws
아산.csv 결측치 처리 완료, 저장된 변수명: asan_aws
유구.csv 결측치 처리 완료, 저장된 변수명: yougu_aws
세종연서.csv 결측치 처리 완료, 저장된 변수명: sjyeonseo_aws
세천.csv 결측치 처리 완료, 저장된 변수명: secheon_aws
세종금남.csv 결측치 처리 완료, 저장된 변수명: sjgeumnam_aws
오월드.csv 결측치 처리 완료, 저장된 변수명: oworld

In [ ]:
TRAIN_AWS_lst = [seocheon_aws, northkyeok_aws, daesan_aws, gyeryong_aws, gongju_aws, seonggeo_aws, 
                 daecheonh_aws, sjgoun_aws, nonsan_aws, dangjin_aws, ongdo_aws, jeongsan_aws, jangdong_aws, 
                 jeongan_aws, yanghwa_aws, yesan_aws, sjjeonui_aws, asan_aws, yougu_aws, sjyeonseo_aws, 
                 secheon_aws, sjgeumnam_aws, oworld_aws, ando_aws, cheongyang_aws, hodo_aws, taean_aws, 
                 hongseongjukdo_aws, chunjang_aws, hongbuk_aws]

In [ ]:
def fill_missing_pm(df): #TRAIN_PM 파일 결측치 처리하는 함수 정의
    df_loc = df.iloc[:, 3:] #연도, 일시, 측정소를 제외한 부분
  
    imputer = KNNImputer(n_neighbors = 50)  
    df_fillna = imputer.fit_transform(df_loc)
    df_fillna = pd.DataFrame(df_fillna, columns=df_loc.columns)
    return df_fillna #결측치 처리

input_folder = "/content/drive/MyDrive/TNT공모전/dataset/TRAIN"  #폴더 경로를 입력

# 파일명-변수명 매핑 딕셔너리
file_pm_variable_dic = {"공주.csv": "gongju_pm", "노은동.csv": "noeun_pm", "논산.csv": "nonsan_pm", "대천2동.csv": "daecheon2_pm", "독곶리.csv": "dokgot_pm", "동문동.csv": "dongmun_pm", 
                     "모종동.csv": "mojong_pm", "문창동.csv": "munchang_pm", "성성동.csv": "seongseong_pm", "신방동.csv": "sinbang_pm", "신흥동.csv": "sinheung_pm", "아름동.csv": "areum_pm", 
                     "예산군.csv": "yesan_pm", "읍내동.csv": "eupnae_pm", "이원면.csv": "iwon_pm", "정림동.csv": "jeonglim_pm", "홍성읍.csv": "hongseong_pm"}


csv_files = [f for f in os.listdir(input_folder) if f.endswith('.csv')] 

# 각 CSV 파일에 대해 결측치 처리 및 동적 변수에 저장
for file in csv_files:
    if file in file_pm_variable_dic:
        file_path = os.path.join(input_folder, file)
        var_pm_train_name = file_pm_variable_dic[file]

        df = pd.read_csv(file_path)
        df_filled = fill_missing_pm(df)
        df_first = df.iloc[:, :3] #일시, 연도, 지점 부분
        result_df = pd.concat([df_first, df_filled], axis=1) #일시, 연도, 지점 부분과 뒷 칼럼 부분을 다시 연결


        # 처리된 데이터프레임을 동적으로 생성된 변수에 저장
        globals()[var_pm_train_name] = result_df

        print(f"{file} 결측치 처리 완료, 저장된 변수명: {var_pm_train_name}")
    else:
        print(f"{file}에 대한 변수명 매핑이 없습니다. 딕셔너리에 추가해 주세요.")

아름동.csv 결측치 처리 완료, 저장된 변수명: areum_pm
논산.csv 결측치 처리 완료, 저장된 변수명: nonsan_pm
이원면.csv 결측치 처리 완료, 저장된 변수명: iwon_pm
공주.csv 결측치 처리 완료, 저장된 변수명: gongju_pm
동문동.csv 결측치 처리 완료, 저장된 변수명: dongmun_pm
예산군.csv 결측치 처리 완료, 저장된 변수명: yesan_pm
대천2동.csv 결측치 처리 완료, 저장된 변수명: daecheon2_pm
홍성읍.csv 결측치 처리 완료, 저장된 변수명: hongseong_pm
성성동.csv 결측치 처리 완료, 저장된 변수명: seongseong_pm
모종동.csv 결측치 처리 완료, 저장된 변수명: mojong_pm
문창동.csv 결측치 처리 완료, 저장된 변수명: munchang_pm
노은동.csv 결측치 처리 완료, 저장된 변수명: noeun_pm
신방동.csv 결측치 처리 완료, 저장된 변수명: sinbang_pm
읍내동.csv 결측치 처리 완료, 저장된 변수명: eupnae_pm
신흥동.csv 결측치 처리 완료, 저장된 변수명: sinheung_pm
독곶리.csv 결측치 처리 완료, 저장된 변수명: dokgot_pm
정림동.csv 결측치 처리 완료, 저장된 변수명: jeonglim_pm


In [ ]:
TRAIN_PM_lst = [areum_pm, nonsan_pm, iwon_pm, gongju_pm, dongmun_pm, yesan_pm, daecheon2_pm, hongseong_pm, 
                seongseong_pm, mojong_pm, munchang_pm, noeun_pm, sinbang_pm, eupnae_pm, sinheung_pm, dokgot_pm, dokgot_pm]

TRAIN 데이터 필요한 칼럼만 처리, 다중선형회귀 모델의 TRAIN 데이터 준비

In [ ]:
#aws train 데이터에서 연도, 지점을 제거하는 코드 (일시만 남음)
dropped_TRAIN_AWS_name = ['gyeryong_aws_train', 'gongju_aws_train', 'nonsan_aws_train', 'dangjin_aws_train', 
                         'daesan_aws_train', 'daecheonh_aws_train', 'northkyeok_aws_train', 'seocheon_aws_train', 
                         'seonggeo_aws_train', 'sjgoun_aws_train', 'sjgeumnam_aws_train', 'sjyeonseo_aws_train', 
                         'sjjeonui_aws_train', 'secheon_aws_train', 'asan_aws_train', 'ando_aws_train', 'yanghwa_aws_train', 
                         'yesan_aws_train', 'oworld_aws_train', 'ongdo_aws_train', 'yougu_aws_train', 'jangdong_aws_train', 
                         'jeongsan_aws_train', 'jeongan_aws_train', 'cheongyang_aws_train', 'chunjang_aws_train', 
                         'taean_aws_train', 'hodo_aws_train', 'hongbuk_aws_train', 'hongseongjukdo_aws_train']

for loc, name in zip(TRAIN_AWS_lst, dropped_TRAIN_AWS_name):
    locals()[name] = loc.drop(columns=['연도', '지점'])

In [ ]:
#pm train 데이터에서 연도, 측정소 제거하는 코드 (일시만 남음)
dropped_TRAIN_PM_name = ['gongju_pm_train', 'noeun_pm_train', 'nonsan_pm_train', 'daecheon2_pm_train', 'dokgot_pm_train', 'dongmun_pm_train', 
                         'mojong_pm_train', 'munchang_pm_train', 'seongseong_pm_train', 'sinbang_pm_train', 'sinheung_pm_train', 
                         'areum_pm_train', 'yesan_pm_train', 'eupnae_pm_train', 'iwon_pm_train', 'jeonglim_pm_train', 'hongseong_pm_train']

for loc, name in zip(TRAIN_PM_lst, dropped_TRAIN_PM_name):
    locals()[name] = loc.drop(columns=['연도', '측정소'])

In [ ]:
#각각 mapping된 aws와 pm 을 연결하여 train 데이터로 만들기
areum_train = pd.merge(sjgoun_aws_train, areum_pm_train, on=['일시'], how='inner')
sinheung_train = pd.merge(sjyeonseo_aws_train, sinheung_pm_train, on=['일시'], how='inner')
noeun_train = pd.merge(gyeryong_aws_train, noeun_pm_train, on=['일시'], how='inner')
munchang_train = pd.merge(oworld_aws_train, munchang_pm_train, on=['일시'], how='inner')
eupnae_train = pd.merge(secheon_aws_train, eupnae_pm_train, on=['일시'], how='inner')
jeonglim_train = pd.merge(oworld_aws_train, jeonglim_pm_train, on=['일시'], how='inner')
gongju_train = pd.merge(gongju_aws_train, gongju_pm_train, on=['일시'], how='inner')
nonsan_train = pd.merge(nonsan_aws_train, nonsan_pm_train, on=['일시'], how='inner')
daecheon2_train = pd.merge(daecheonh_aws_train, daecheon2_pm_train, on=['일시'], how='inner')
dokgot_train = pd.merge(daesan_aws_train, dokgot_pm_train, on=['일시'], how='inner')
dongmun_train = pd.merge(taean_aws_train, dongmun_pm_train, on=['일시'], how='inner')
mojong_train = pd.merge(asan_aws_train, mojong_pm_train, on=['일시'], how='inner')
sinbang_train = pd.merge(sjjeonui_aws_train, sinbang_pm_train, on=['일시'], how='inner')
yesan_train = pd.merge(yesan_aws_train, yesan_pm_train, on=['일시'], how='inner')
iwon_train = pd.merge(taean_aws_train, iwon_pm_train, on=['일시'], how='inner')
hongseong_train = pd.merge(hongbuk_aws_train, hongseong_pm_train, on=['일시'], how='inner')
seongseong_train = pd.merge(seonggeo_aws_train, seongseong_pm_train, on=['일시'], how='inner')

In [ ]:
train_data = pd.concat([gongju_train, noeun_train, nonsan_train, daecheon2_train, 
                        dokgot_train, dongmun_train, mojong_train, munchang_train, 
                        seongseong_train, sinbang_train, sinheung_train, areum_train, 
                        yesan_train, eupnae_train, iwon_train, jeonglim_train, 
                        hongseong_train], ignore_index=True)

In [ ]:
train_data.columns = ['date', 'temp', 'wind_d', 'wind_s', 'prec', 'humid', 'PM2.5']
train_data

,date,temp,wind_d,wind_s,prec,humid,PM2.5
0,01-01 00:00,0.273302,0.915556,0.255754,0.0,0.850,0.044
1,01-01 00:00,0.273302,0.915556,0.255754,0.0,0.850,0.312
2,01-01 00:00,0.273302,0.915556,0.255754,0.0,0.850,0.128
3,01-01 00:00,0.273302,0.915556,0.255754,0.0,0.850,0.092
4,01-01 00:00,0.436019,0.030833,0.053708,0.0,0.780,0.044
...,...,...,...,...,...,...,...
2383123,02-29 19:00,0.519747,0.965278,0.094629,0.0,0.750,0.196
2383124,02-29 20:00,0.524487,0.972778,0.071611,0.0,0.728,0.220
2383125,02-29 21:00,0.518167,0.934722,0.048593,0.0,0.712,0.272
2383126,02-29 22:00,0.499210,0.868889,0.058824,0.0,0.812,0.280


다중선형회귀모델

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

# train set, test set 준비
train_set = train_data.drop(columns=['date'])

x_train = train_set.drop(columns=['PM2.5'])
y_train = train_set['PM2.5']

interaction_features = x_train[['temp', 'wind_d', 'prec']]
non_interaction = x_train[['wind_s', 'humid']]
interaction = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)
X_interaction = interaction.fit_transform(interaction_features)

X_combined = np.hstack([X_interaction, non_interaction])

#모델 학습
lr = LinearRegression()
model = lr.fit(X_combined, y_train)


# 3. 학습된 모델을 사용하여 잔차 계산
y_pred = model.predict(X_combined)
residuals = y_train - y_pred

# 4. 잔차를 기반으로 이상치 찾기 (IQR 방법 사용)
Q1 = residuals.quantile(0.25)
Q3 = residuals.quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = (residuals < lower_bound) | (residuals > upper_bound)

# 5. 이상치 제거
x_train_no_outliers = X_combined[~outliers]
y_train_no_outliers = y_train[~outliers]

# 6. 이상치가 제거된 데이터로 선형회귀 모델을 다시 학습
model_no_outliers = LinearRegression().fit(x_train_no_outliers, y_train_no_outliers)

In [ ]:
# 회귀계수 추출
coefficients = model_no_outliers.coef_

# 회귀계수와 독립 변수 이름을 함께 출력
for feature, coef in zip(x_train.columns, coefficients):
    print(f"{feature}: {coef}")

temp: -0.08540369413586303
wind_d: -0.0008733236627136553
wind_s: -0.06213756620288156
prec: -0.07408503316987433
humid: -0.010717945160066295


In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

# train set, test set 준비
train_set = train_data.drop(columns=['date'])

x_train = train_set.drop(columns=['PM2.5'])
y_train = train_set['PM2.5']

#모델 학습
lr = LinearRegression()
model = lr.fit(x_train, y_train)

In [ ]:
# 회귀계수 추출
coefficients = lr.coef_

# 회귀계수와 독립 변수 이름을 함께 출력
for feature, coef in zip(x_train.columns, coefficients):
    print(f"{feature}: {coef}")

temp: -0.12190416189577519
wind_d: -0.0007580826708504622
wind_s: -0.0655623515571165
prec: -0.07245800889544303
humid: -0.014838708456122568


In [ ]:
import statsmodels.api as sm


X = sm.add_constant(x_train)
model = sm.OLS(y_train, X).fit()

print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  PM2.5   R-squared:                       0.084
Model:                            OLS   Adj. R-squared:                  0.084
Method:                 Least Squares   F-statistic:                 4.364e+04
Date:                Thu, 11 May 2023   Prob (F-statistic):               0.00
Time:                        01:24:57   Log-Likelihood:             3.0171e+06
No. Observations:             2383128   AIC:                        -6.034e+06
Df Residuals:                 2383122   BIC:                        -6.034e+06
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.1706      0.000    667.509      0.0

RNN 모델로 AWS 예측하여 3일치 기상데이터 채우기

In [ ]:
filename_to_var = {"계룡.csv": "gyeryong_aws_t", "공주.csv": "gongju_aws_t", "논산.csv": "nonsan_aws_t", "당진.csv": "dangjin_aws_t", "대산.csv": "daesan_aws_t", 
                      "대천항.csv": "daecheonh_aws_t", "북격렬비도.csv": "northkyeok_aws_t", "서천.csv": "seocheon_aws_t", "성거.csv": "seonggeo_aws_t", 
                      "세종고운.csv": "sjgoun_aws_t", "세종금남.csv": "sjgeumnam_aws_t", "세종연서.csv": "sjyeonseo_aws_t", "세종전의.csv": "sjjeonui_aws_t", 
                      "세천.csv": "secheon_aws_t", "아산.csv": "asan_aws_t", "안도.csv": "ando_aws_t", "양화.csv": "yanghwa_aws_t", "예산.csv": "yesan_aws_t", 
                      "오월드.csv":"oworld_aws_t", "옹도.csv": "ongdo_aws_t", "유구.csv": "yougu_aws_t", "장동.csv": "jangdong_aws_t", "정산.csv": "jeongsan_aws_t", 
                      "정안.csv": "jeongan_aws_t", "청양.csv": "cheongyang_aws_t", "춘장대.csv": "chunjang_aws_t", "태안.csv": "taean_aws_t", "호도.csv": "hodo_aws_t", 
                      "홍북.csv": "hongbuk_aws_t", "홍성죽도.csv": "hongseongjukdo_aws_t"}

folder_path = "/content/drive/MyDrive/TNT공모전/dataset/TEST_AWS"

for filename, variable_name in filename_to_var.items():
    file_path = f"{folder_path}/{filename}"
    globals()[variable_name] = pd.read_csv(file_path, encoding='utf-8')

    print(f"{filename} 저장된 변수명: {variable_name}")

계룡.csv 저장된 변수명: gyeryong_aws_t
공주.csv 저장된 변수명: gongju_aws_t
논산.csv 저장된 변수명: nonsan_aws_t
당진.csv 저장된 변수명: dangjin_aws_t
대산.csv 저장된 변수명: daesan_aws_t
대천항.csv 저장된 변수명: daecheonh_aws_t
북격렬비도.csv 저장된 변수명: northkyeok_aws_t
서천.csv 저장된 변수명: seocheon_aws_t
성거.csv 저장된 변수명: seonggeo_aws_t
세종고운.csv 저장된 변수명: sjgoun_aws_t
세종금남.csv 저장된 변수명: sjgeumnam_aws_t
세종연서.csv 저장된 변수명: sjyeonseo_aws_t
세종전의.csv 저장된 변수명: sjjeonui_aws_t
세천.csv 저장된 변수명: secheon_aws_t
아산.csv 저장된 변수명: asan_aws_t
안도.csv 저장된 변수명: ando_aws_t
양화.csv 저장된 변수명: yanghwa_aws_t
예산.csv 저장된 변수명: yesan_aws_t
오월드.csv 저장된 변수명: oworld_aws_t
옹도.csv 저장된 변수명: ongdo_aws_t
유구.csv 저장된 변수명: yougu_aws_t
장동.csv 저장된 변수명: jangdong_aws_t
정산.csv 저장된 변수명: jeongsan_aws_t
정안.csv 저장된 변수명: jeongan_aws_t
청양.csv 저장된 변수명: cheongyang_aws_t
춘장대.csv 저장된 변수명: chunjang_aws_t
태안.csv 저장된 변수명: taean_aws_t
호도.csv 저장된 변수명: hodo_aws_t
홍북.csv 저장된 변수명: hongbuk_aws_t
홍ᄉ

In [ ]:
TEST_AWS_lst = [seocheon_aws_t, northkyeok_aws_t, daesan_aws_t, gyeryong_aws_t, gongju_aws_t, seonggeo_aws_t, 
                 daecheonh_aws_t, sjgoun_aws_t, nonsan_aws_t, dangjin_aws_t, ongdo_aws_t, jeongsan_aws_t, jangdong_aws_t, 
                 jeongan_aws_t, yanghwa_aws_t, yesan_aws_t, sjjeonui_aws_t, asan_aws_t, yougu_aws_t, sjyeonseo_aws_t, 
                 secheon_aws_t, sjgeumnam_aws_t, oworld_aws_t, ando_aws_t, cheongyang_aws_t, hodo_aws_t, taean_aws_t, 
                 hongseongjukdo_aws_t, chunjang_aws_t, hongbuk_aws_t]

In [ ]:
def RNN_AWS_predict(data_a, data_b):
  data_a_drop = data_a.drop(columns=['일시', '연도', '지점'])
  data_b_drop = data_b.drop(columns=['일시', '연도', '지점'])
  a_values = data_a_drop.values.astype('float32')
  b_values = data_b_drop.values.astype('float32')


  # 시계열 데이터 생성
  look_back = 2
  train_generator = TimeseriesGenerator(a_values, a_values, length=look_back, batch_size=1)

  # RNN 모델 정의
  model = Sequential()
  model.add(SimpleRNN(128, input_shape=(look_back, 5)))
  model.add(Dense(5))
  model.compile(loss='mean_squared_error', optimizer='nadam')

  model.fit_generator(train_generator, epochs=10, steps_per_epoch=len(train_generator))

  # b 파일에서 비어 있는 열 채우기
  n_fill_columns = 4608
  filled_data = b_values.copy()

  # 비어 있는 열에 대한 인덱스를 찾습니다.
  empty_indices = np.where(pd.isnull(data_b_drop))[0]
  null_index = np.unique(empty_indices)

  for i in range(0, n_fill_columns, 3):
    # 비어 있는 열 바로 앞의 2일치 데이터를 가져옵니다.
    input_data = filled_data[null_index[i]-2:null_index[i], :].reshape(1, look_back, 5)
    
    # 3일치 데이터를 예측합니다.
    prediction = model.predict(input_data)
    
    # 비어 있는 열에 예측값을 채웁니다.
    filled_data[null_index[i]:null_index[i]+3, :] = prediction

  # 예측 결과 저장
  filled_b_data = pd.DataFrame(filled_data, columns=data_b_drop.columns)
  return filled_b_data

In [ ]:
#세종고운 TEST_AWS 3일치 데이터 채우기
filled_sjgoun_aws = RNN_AWS_predict(sjgoun_aws, sjgoun_aws_t)
filled_sjgoun_aws

Epoch 1/10


<ipython-input-24-1380cf1a8bad>:18: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, epochs=10, steps_per_epoch=len(train_generator))


35062/35062 [==============================] - 100s 3ms/step - loss: 0.0146
Epoch 2/10
35062/35062 [==============================] - 104s 3ms/step - loss: 0.0139
Epoch 3/10
35062/35062 [==============================] - 103s 3ms/step - loss: 0.0138
Epoch 4/10
35062/35062 [==============================] - 101s 3ms/step - loss: 0.0137
Epoch 5/10
35062/35062 [==============================] - 101s 3ms/step - loss: 0.0136
Epoch 6/10
35062/35062 [==============================] - 99s 3ms/step - loss: 0.0135
Epoch 7/10
35062/35062 [==============================] - 106s 3ms/step - loss: 0.0134
Epoch 8/10
35062/35062 [==============================] - 101s 3ms/step - loss: 0.0134
Epoch 9/10
35062/35062 [==============================] - 100s 3ms/step - loss: 0.0134
Epoch 10/10
1/1 [==============================] - 0s 25ms/step


,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%)
0,0.232227,0.091944,0.012788,0.0,0.683
1,0.225908,0.127222,0.012788,0.0,0.633
2,0.222749,0.000000,0.002558,0.0,0.630
3,0.211690,0.000000,0.000000,0.0,0.641
4,0.218009,0.000000,0.005115,0.0,0.652
...,...,...,...,...,...
7723,0.429700,0.808889,0.066496,0.0,0.446
7724,0.417062,0.835000,0.109974,0.0,0.510
7725,0.399684,0.816944,0.094629,0.0,0.595
7726,0.390205,0.816111,0.056266,0.0,0.643


In [ ]:
 filled_sjgoun_aws.to_csv('/content/drive//MyDrive/TNT공모전/test_aws/sjgoun_aws.csv', index=False, encoding='utf-8-sig')

In [ ]:
#세종연서 TEST_AWS 3일치 데이터 채우기
filled_sjyeonseo_aws = RNN_AWS_predict(sjyeonseo_aws, sjyeonseo_aws_t)
filled_sjyeonseo_aws

Epoch 1/10


<ipython-input-22-1380cf1a8bad>:18: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, epochs=10, steps_per_epoch=len(train_generator))


35062/35062 [==============================] - 120s 3ms/step - loss: 0.0245
Epoch 2/10
35062/35062 [==============================] - 111s 3ms/step - loss: 0.0235
Epoch 3/10
35062/35062 [==============================] - 104s 3ms/step - loss: 0.0234
Epoch 4/10
35062/35062 [==============================] - 105s 3ms/step - loss: 0.0232
Epoch 5/10
35062/35062 [==============================] - 114s 3ms/step - loss: 0.0230
Epoch 6/10
35062/35062 [==============================] - 102s 3ms/step - loss: 0.0229
Epoch 7/10
35062/35062 [==============================] - 102s 3ms/step - loss: 0.0228
Epoch 8/10
35062/35062 [==============================] - 103s 3ms/step - loss: 0.0228
Epoch 9/10
35062/35062 [==============================] - 111s 3ms/step - loss: 0.0227
Epoch 10/10
1/1 [==============================] - 0s 22ms/step


In [ ]:
filled_sjyeonseo_aws.to_csv('/content/drive//MyDrive/TNT공모전/test_aws/sjyeonseo_aws.csv', index=False, encoding='utf-8-sig')

In [ ]:
#계룡 TEST_AWS 3일치 데이터 채우기
filled_gyeryong_aws = RNN_AWS_predict(gyeryong_aws, gyeryong_aws_t)
filled_gyeryong_aws

Epoch 1/10


<ipython-input-48-1380cf1a8bad>:18: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, epochs=10, steps_per_epoch=len(train_generator))


35062/35062 [==============================] - 110s 3ms/step - loss: 0.0141
Epoch 2/10
35062/35062 [==============================] - 104s 3ms/step - loss: 0.0132
Epoch 3/10
35062/35062 [==============================] - 108s 3ms/step - loss: 0.0131
Epoch 4/10
35062/35062 [==============================] - 106s 3ms/step - loss: 0.0130
Epoch 5/10
35062/35062 [==============================] - 107s 3ms/step - loss: 0.0129
Epoch 6/10
35062/35062 [==============================] - 99s 3ms/step - loss: 0.0128
Epoch 7/10
35062/35062 [==============================] - 102s 3ms/step - loss: 0.0127
Epoch 8/10
35062/35062 [==============================] - 108s 3ms/step - loss: 0.0127
Epoch 9/10
35062/35062 [==============================] - 115s 3ms/step - loss: 0.0126
Epoch 10/10
1/1 [==============================] - 0s 36ms/step


,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%)
0,0.244866,0.281667,0.020460,0.0,0.622
1,0.224329,0.365833,0.015345,0.0,0.691
2,0.195893,0.000000,0.010230,0.0,0.772
3,0.173776,0.223333,0.015345,0.0,0.808
4,0.164297,0.000000,0.005115,0.0,0.812
...,...,...,...,...,...
7723,0.437599,0.714722,0.053708,0.0,0.384
7724,0.426540,0.741944,0.066496,0.0,0.414
7725,0.409163,0.767222,0.079284,0.0,0.490
7726,0.396524,0.824444,0.058824,0.0,0.534


In [ ]:
filled_gyeryong_aws.to_csv('/content/drive//MyDrive/TNT공모전/test_aws/gyeryong_aws.csv', index=False, encoding='utf-8-sig')

In [ ]:
#오월드 TEST_AWS 3일치 데이터 채우기
filled_oworld_aws = RNN_AWS_predict(oworld_aws, oworld_aws_t)
filled_oworld_aws

Epoch 1/10


<ipython-input-48-1380cf1a8bad>:18: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, epochs=10, steps_per_epoch=len(train_generator))


35062/35062 [==============================] - 111s 3ms/step - loss: 0.0154
Epoch 2/10
35062/35062 [==============================] - 105s 3ms/step - loss: 0.0146
Epoch 3/10
35062/35062 [==============================] - 103s 3ms/step - loss: 0.0145
Epoch 4/10
35062/35062 [==============================] - 106s 3ms/step - loss: 0.0145
Epoch 5/10
35062/35062 [==============================] - 102s 3ms/step - loss: 0.0144
Epoch 6/10
35062/35062 [==============================] - 108s 3ms/step - loss: 0.0143
Epoch 7/10
35062/35062 [==============================] - 104s 3ms/step - loss: 0.0142
Epoch 8/10
35062/35062 [==============================] - 111s 3ms/step - loss: 0.0142
Epoch 9/10
35062/35062 [==============================] - 112s 3ms/step - loss: 0.0141
Epoch 10/10
1/1 [==============================] - 0s 37ms/step


,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%)
0,0.241706,0.096944,0.046036,0.0,0.718
1,0.229068,0.141667,0.020460,0.0,0.690
2,0.230648,0.000000,0.002558,0.0,0.658
3,0.205371,0.000000,0.005115,0.0,0.692
4,0.202212,0.452222,0.043478,0.0,0.785
...,...,...,...,...,...
7723,0.445498,0.796944,0.071611,0.0,0.257
7724,0.434439,0.805278,0.099744,0.0,0.299
7725,0.413902,0.860278,0.086957,0.0,0.457
7726,0.396524,0.869722,0.074169,0.0,0.491


In [ ]:
filled_oworld_aws.to_csv('/content/drive//MyDrive/TNT공모전/test_aws/oworld_aws.csv', index=False, encoding='utf-8-sig')

In [ ]:
#세천 TEST_AWS 3일치 데이터 채우기
filled_secheon_aws = RNN_AWS_predict(secheon_aws, secheon_aws_t)
filled_secheon_aws

Epoch 1/10


<ipython-input-48-1380cf1a8bad>:18: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, epochs=10, steps_per_epoch=len(train_generator))


35062/35062 [==============================] - 121s 3ms/step - loss: 0.0170
Epoch 2/10
35062/35062 [==============================] - 118s 3ms/step - loss: 0.0160
Epoch 3/10
35062/35062 [==============================] - 122s 3ms/step - loss: 0.0158
Epoch 4/10
35062/35062 [==============================] - 123s 4ms/step - loss: 0.0158
Epoch 5/10
35062/35062 [==============================] - 125s 4ms/step - loss: 0.0157
Epoch 6/10
35062/35062 [==============================] - 120s 3ms/step - loss: 0.0157
Epoch 7/10
35062/35062 [==============================] - 118s 3ms/step - loss: 0.0157
Epoch 8/10
35062/35062 [==============================] - 117s 3ms/step - loss: 0.0156
Epoch 9/10
35062/35062 [==============================] - 119s 3ms/step - loss: 0.0156
Epoch 10/10
1/1 [==============================] - 0s 27ms/step


,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%)
0,0.241706,0.506389,0.046036,0.0,0.655
1,0.227488,0.454444,0.028133,0.0,0.652
2,0.216430,0.000000,0.007673,0.0,0.662
3,0.194313,0.000000,0.007673,0.0,0.721
4,0.187994,0.000000,0.005115,0.0,0.757
...,...,...,...,...,...
7723,0.454976,0.446389,0.066496,0.0,0.328
7724,0.439179,0.562500,0.061381,0.0,0.330
7725,0.421801,0.523056,0.081841,0.0,0.405
7726,0.404423,0.522778,0.076726,0.0,0.509


In [ ]:
filled_secheon_aws.to_csv('/content/drive//MyDrive/TNT공모전/test_aws/secheon_aws.csv', index=False, encoding='utf-8-sig')

In [ ]:
#공주 TEST_AWS 3일치 데이터 채우기
filled_gongju_aws = RNN_AWS_predict(gongju_aws, gongju_aws_t)
filled_gongju_aws

Epoch 1/10


<ipython-input-22-1380cf1a8bad>:18: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, epochs=10, steps_per_epoch=len(train_generator))


35062/35062 [==============================] - 110s 3ms/step - loss: 0.0159
Epoch 2/10
35062/35062 [==============================] - 96s 3ms/step - loss: 0.0153
Epoch 3/10
35062/35062 [==============================] - 91s 3ms/step - loss: 0.0152
Epoch 4/10
35062/35062 [==============================] - 89s 3ms/step - loss: 0.0150
Epoch 5/10
35062/35062 [==============================] - 89s 3ms/step - loss: 0.0149
Epoch 6/10
35062/35062 [==============================] - 89s 3ms/step - loss: 0.0147
Epoch 7/10
35062/35062 [==============================] - 92s 3ms/step - loss: 0.0146
Epoch 8/10
35062/35062 [==============================] - 89s 3ms/step - loss: 0.0146
Epoch 9/10
35062/35062 [==============================] - 90s 3ms/step - loss: 0.0144
Epoch 10/10
1/1 [==============================] - 0s 33ms/step


,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%)
0,0.244866,0.123333,0.038363,0.0,0.647
1,0.232227,0.167778,0.033248,0.0,0.648
2,0.206951,0.000000,0.002558,0.0,0.734
3,0.199052,0.000000,0.002558,0.0,0.753
4,0.189573,0.000000,0.002558,0.0,0.795
...,...,...,...,...,...
7723,0.445498,0.791667,0.097187,0.0,0.368
7724,0.426540,0.781667,0.104859,0.0,0.502
7725,0.413902,0.735278,0.048593,0.0,0.484
7726,0.404423,0.727222,0.058824,0.0,0.564


In [ ]:
filled_gongju_aws.to_csv('/content/drive//MyDrive/TNT공모전/test_aws/gongju_aws.csv', index=False, encoding='utf-8-sig')

In [ ]:
#논산 TEST_AWS 3일치 데이터 채우기
filled_nonsan_aws = RNN_AWS_predict(nonsan_aws, nonsan_aws_t)
filled_nonsan_aws

Epoch 1/10


<ipython-input-22-1380cf1a8bad>:18: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, epochs=10, steps_per_epoch=len(train_generator))


35062/35062 [==============================] - 95s 3ms/step - loss: 0.0161
Epoch 2/10
35062/35062 [==============================] - 95s 3ms/step - loss: 0.0153
Epoch 3/10
35062/35062 [==============================] - 93s 3ms/step - loss: 0.0150
Epoch 4/10
35062/35062 [==============================] - 93s 3ms/step - loss: 0.0148
Epoch 5/10
35062/35062 [==============================] - 92s 3ms/step - loss: 0.0147
Epoch 6/10
35062/35062 [==============================] - 96s 3ms/step - loss: 0.0146
Epoch 7/10
35062/35062 [==============================] - 93s 3ms/step - loss: 0.0146
Epoch 8/10
35062/35062 [==============================] - 96s 3ms/step - loss: 0.0145
Epoch 9/10
35062/35062 [==============================] - 96s 3ms/step - loss: 0.0145
Epoch 10/10
1/1 [==============================] - 0s 29ms/step


,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%)
0,0.263823,0.009722,0.038363,0.0,0.683
1,0.246445,0.175556,0.025575,0.0,0.746
2,0.230648,0.150833,0.015345,0.0,0.805
3,0.227488,0.183056,0.020460,0.0,0.818
4,0.219589,0.139444,0.028133,0.0,0.818
...,...,...,...,...,...
7723,0.451817,0.855833,0.107417,0.0,0.390
7724,0.440758,0.806944,0.115090,0.0,0.548
7725,0.426540,0.820278,0.089514,0.0,0.550
7726,0.417062,0.907222,0.089514,0.0,0.524


In [ ]:
filled_nonsan_aws.to_csv('/content/drive//MyDrive/TNT공모전/test_aws/nonsan_aws.csv', index=False, encoding='utf-8-sig')

In [ ]:
#대천항 TEST_AWS 3일치 데이터 채우기
filled_daecheonh_aws = RNN_AWS_predict(daecheonh_aws, daecheonh_aws_t)
filled_daecheonh_aws

Epoch 1/10


<ipython-input-22-1380cf1a8bad>:18: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, epochs=10, steps_per_epoch=len(train_generator))


35062/35062 [==============================] - 101s 3ms/step - loss: 0.0122
Epoch 2/10
35062/35062 [==============================] - 95s 3ms/step - loss: 0.0116
Epoch 3/10
35062/35062 [==============================] - 96s 3ms/step - loss: 0.0114
Epoch 4/10
35062/35062 [==============================] - 96s 3ms/step - loss: 0.0113
Epoch 5/10
35062/35062 [==============================] - 98s 3ms/step - loss: 0.0112
Epoch 6/10
35062/35062 [==============================] - 97s 3ms/step - loss: 0.0112
Epoch 7/10
35062/35062 [==============================] - 96s 3ms/step - loss: 0.0111
Epoch 8/10
35062/35062 [==============================] - 95s 3ms/step - loss: 0.0111
Epoch 9/10
35062/35062 [==============================] - 95s 3ms/step - loss: 0.0110
Epoch 10/10
1/1 [==============================] - 0s 21ms/step


,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%)
0,0.285940,0.092778,0.127877,0.0,0.730
1,0.284360,0.113333,0.143223,0.0,0.727
2,0.287520,0.070278,0.148338,0.0,0.747
3,0.284360,0.113611,0.115090,0.0,0.728
4,0.285940,0.113333,0.125320,0.0,0.739
...,...,...,...,...,...
7723,0.440758,0.940278,0.312020,0.0,0.600
7724,0.437599,0.960000,0.322251,0.0,0.736
7725,0.432859,0.944722,0.355499,0.0,0.747
7726,0.424960,0.957222,0.324808,0.0,0.755


In [ ]:
filled_daecheonh_aws.to_csv('/content/drive//MyDrive/TNT공모전/test_aws/daecheonh_aws.csv', index=False, encoding='utf-8-sig')

In [ ]:
#대산 TEST_AWS 3일치 데이터 채우기
filled_daesan_aws = RNN_AWS_predict(daesan_aws, daesan_aws_t)
filled_daesan_aws

Epoch 1/10


<ipython-input-22-1380cf1a8bad>:18: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, epochs=10, steps_per_epoch=len(train_generator))


35062/35062 [==============================] - 94s 3ms/step - loss: 0.0123
Epoch 2/10
35062/35062 [==============================] - 92s 3ms/step - loss: 0.0116
Epoch 3/10
35062/35062 [==============================] - 91s 3ms/step - loss: 0.0115
Epoch 4/10
35062/35062 [==============================] - 92s 3ms/step - loss: 0.0113
Epoch 5/10
35062/35062 [==============================] - 93s 3ms/step - loss: 0.0113
Epoch 6/10
35062/35062 [==============================] - 95s 3ms/step - loss: 0.0112
Epoch 7/10
35062/35062 [==============================] - 94s 3ms/step - loss: 0.0112
Epoch 8/10
35062/35062 [==============================] - 94s 3ms/step - loss: 0.0111
Epoch 9/10
35062/35062 [==============================] - 93s 3ms/step - loss: 0.0111
Epoch 10/10
1/1 [==============================] - 0s 22ms/step


,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%)
0,0.303318,0.935000,0.089514,0.0,0.656
1,0.303318,0.928889,0.102302,0.0,0.601
2,0.303318,0.922500,0.097187,0.0,0.630
3,0.301738,0.942778,0.094629,0.0,0.612
4,0.296998,0.902778,0.084399,0.0,0.634
...,...,...,...,...,...
7723,0.421801,0.742778,0.104859,0.0,0.624
7724,0.415482,0.662222,0.071611,0.0,0.686
7725,0.406003,0.696111,0.104859,0.0,0.698
7726,0.388626,0.595278,0.102302,0.0,0.654


In [ ]:
filled_daesan_aws.to_csv('/content/drive//MyDrive/TNT공모전/test_aws/daesan_aws.csv', index=False, encoding='utf-8-sig')

In [ ]:
#태안 TEST_AWS 3일치 데이터 채우기
filled_taean_aws = RNN_AWS_predict(taean_aws, taean_aws_t)
filled_taean_aws

Epoch 1/10


<ipython-input-22-1380cf1a8bad>:18: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, epochs=10, steps_per_epoch=len(train_generator))


35062/35062 [==============================] - 95s 3ms/step - loss: 0.0149
Epoch 2/10
35062/35062 [==============================] - 91s 3ms/step - loss: 0.0143
Epoch 3/10
35062/35062 [==============================] - 91s 3ms/step - loss: 0.0141
Epoch 4/10
35062/35062 [==============================] - 90s 3ms/step - loss: 0.0140
Epoch 5/10
35062/35062 [==============================] - 90s 3ms/step - loss: 0.0139
Epoch 6/10
35062/35062 [==============================] - 92s 3ms/step - loss: 0.0138
Epoch 7/10
35062/35062 [==============================] - 92s 3ms/step - loss: 0.0137
Epoch 8/10
35062/35062 [==============================] - 90s 3ms/step - loss: 0.0137
Epoch 9/10
35062/35062 [==============================] - 92s 3ms/step - loss: 0.0136
Epoch 10/10
1/1 [==============================] - 0s 30ms/step


,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%)
0,0.285940,0.140556,0.043478,0.0,0.679
1,0.289100,0.114444,0.040921,0.0,0.693
2,0.300158,0.222222,0.023018,0.0,0.697
3,0.293839,0.128889,0.040921,0.0,0.696
4,0.290679,0.192222,0.030691,0.0,0.708
...,...,...,...,...,...
7723,0.404423,0.885556,0.094629,0.0,0.602
7724,0.404423,0.852778,0.081841,0.0,0.651
7725,0.394945,0.875556,0.104859,0.0,0.633
7726,0.387046,0.866667,0.086957,0.0,0.622


In [ ]:
filled_taean_aws.to_csv('/content/drive//MyDrive/TNT공모전/test_aws/taean_aws.csv', index=False, encoding='utf-8-sig')

In [ ]:
#아산 TEST_AWS 3일치 데이터 채우기
filled_asan_aws = RNN_AWS_predict(asan_aws, asan_aws_t)
filled_asan_aws

Epoch 1/10


<ipython-input-22-1380cf1a8bad>:18: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, epochs=10, steps_per_epoch=len(train_generator))


35062/35062 [==============================] - 95s 3ms/step - loss: 0.0140
Epoch 2/10
35062/35062 [==============================] - 93s 3ms/step - loss: 0.0134
Epoch 3/10
35062/35062 [==============================] - 93s 3ms/step - loss: 0.0132
Epoch 4/10
35062/35062 [==============================] - 92s 3ms/step - loss: 0.0131
Epoch 5/10
35062/35062 [==============================] - 91s 3ms/step - loss: 0.0130
Epoch 6/10
35062/35062 [==============================] - 93s 3ms/step - loss: 0.0130
Epoch 7/10
35062/35062 [==============================] - 93s 3ms/step - loss: 0.0129
Epoch 8/10
35062/35062 [==============================] - 92s 3ms/step - loss: 0.0129
Epoch 9/10
35062/35062 [==============================] - 93s 3ms/step - loss: 0.0128
Epoch 10/10
1/1 [==============================] - 0s 22ms/step


,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%)
0,0.249605,0.141389,0.063939,0.0,0.547
1,0.244866,0.112222,0.063939,0.0,0.547
2,0.240126,0.123056,0.056266,0.0,0.537
3,0.218009,0.385000,0.030691,0.0,0.618
4,0.210111,0.438611,0.023018,0.0,0.726
...,...,...,...,...,...
7723,0.420221,0.816667,0.104859,0.0,0.536
7724,0.406003,0.819722,0.109974,0.0,0.571
7725,0.401264,0.855556,0.145780,0.0,0.584
7726,0.391785,0.803889,0.150895,0.0,0.569


In [ ]:
filled_asan_aws.to_csv('/content/drive//MyDrive/TNT공모전/test_aws/asan_aws.csv', index=False, encoding='utf-8-sig')

In [ ]:
#세종전의 EST_AWS 3일치 데이터 채우기
filled_sjjeonui_aws = RNN_AWS_predict(sjjeonui_aws, sjjeonui_aws_t)
filled_sjjeonui_aws

Epoch 1/10


<ipython-input-22-1380cf1a8bad>:18: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, epochs=10, steps_per_epoch=len(train_generator))


35062/35062 [==============================] - 93s 3ms/step - loss: 0.0191
Epoch 2/10
35062/35062 [==============================] - 92s 3ms/step - loss: 0.0182
Epoch 3/10
35062/35062 [==============================] - 92s 3ms/step - loss: 0.0180
Epoch 4/10
35062/35062 [==============================] - 91s 3ms/step - loss: 0.0179
Epoch 5/10
35062/35062 [==============================] - 92s 3ms/step - loss: 0.0178
Epoch 6/10
35062/35062 [==============================] - 91s 3ms/step - loss: 0.0177
Epoch 7/10
35062/35062 [==============================] - 92s 3ms/step - loss: 0.0176
Epoch 8/10
35062/35062 [==============================] - 91s 3ms/step - loss: 0.0176
Epoch 9/10
35062/35062 [==============================] - 93s 3ms/step - loss: 0.0175
Epoch 10/10
1/1 [==============================] - 0s 23ms/step


,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%)
0,0.208531,0.033333,0.025575,0.0,0.561
1,0.184834,0.000000,0.002558,0.0,0.690
2,0.165877,0.000000,0.002558,0.0,0.758
3,0.157978,0.036667,0.015345,0.0,0.784
4,0.150079,0.000000,0.007673,0.0,0.823
...,...,...,...,...,...
7723,0.424960,0.919167,0.089514,0.0,0.434
7724,0.407583,0.953333,0.107417,0.0,0.547
7725,0.393365,0.902500,0.115090,0.0,0.572
7726,0.388626,0.981111,0.084399,0.0,0.592


In [ ]:
filled_sjjeonui_aws.to_csv('/content/drive//MyDrive/TNT공모전/test_aws/sjjeonui_aws.csv', index=False, encoding='utf-8-sig')

In [ ]:
#예산 TEST_AWS 3일치 데이터 채우기
filled_yesan_aws = RNN_AWS_predict(yesan_aws, yesan_aws_t)
filled_yesan_aws

Epoch 1/10


<ipython-input-22-1380cf1a8bad>:18: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, epochs=10, steps_per_epoch=len(train_generator))


 1692/35062 [>.............................] - ETA: 2:59 - loss: 0.0194

KeyboardInterrupt: ignored

In [ ]:
filled_yesan_aws.to_csv('/content/drive//MyDrive/TNT공모전/test_aws/yesan_aws.csv', index=False, encoding='utf-8-sig')

In [ ]:
#홍북 TEST_AWS 3일치 데이터 채우기
filled_hongbuk_aws = RNN_AWS_predict(hongbuk_aws, hongbuk_aws_t)
filled_hongbuk_aws

Epoch 1/10


<ipython-input-22-1380cf1a8bad>:18: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, epochs=10, steps_per_epoch=len(train_generator))


35062/35062 [==============================] - 104s 3ms/step - loss: 0.0133
Epoch 2/10
35062/35062 [==============================] - 107s 3ms/step - loss: 0.0127
Epoch 3/10
35062/35062 [==============================] - 103s 3ms/step - loss: 0.0125
Epoch 4/10
35062/35062 [==============================] - 102s 3ms/step - loss: 0.0123
Epoch 5/10
35062/35062 [==============================] - 100s 3ms/step - loss: 0.0122
Epoch 6/10
35062/35062 [==============================] - 102s 3ms/step - loss: 0.0121
Epoch 7/10
35062/35062 [==============================] - 103s 3ms/step - loss: 0.0121
Epoch 8/10
35062/35062 [==============================] - 104s 3ms/step - loss: 0.0121
Epoch 9/10
35062/35062 [==============================] - 103s 3ms/step - loss: 0.0120
Epoch 10/10
1/1 [==============================] - 0s 34ms/step


,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%)
0,0.241706,0.305278,0.038363,0.0,0.757
1,0.222749,0.000000,0.005115,0.0,0.763
2,0.222749,0.000000,0.000000,0.0,0.713
3,0.224329,0.000000,0.000000,0.0,0.752
4,0.213270,0.000000,0.000000,0.0,0.760
...,...,...,...,...,...
7723,0.404423,0.833611,0.089514,0.0,0.554
7724,0.396524,0.830556,0.074169,0.0,0.552
7725,0.393365,0.837500,0.084399,0.0,0.579
7726,0.387046,0.806944,0.094629,0.0,0.593


In [ ]:
filled_hongbuk_aws.to_csv('/content/drive//MyDrive/TNT공모전/test_aws/hongbuk_aws.csv', index=False, encoding='utf-8-sig')

In [ ]:
#성거 TEST_AWS 3일치 데이터 채우기
filled_seonggeo_aws = RNN_AWS_predict(seonggeo_aws, seonggeo_aws_t)
filled_seonggeo_aws

Epoch 1/10


<ipython-input-22-1380cf1a8bad>:18: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, epochs=10, steps_per_epoch=len(train_generator))


35062/35062 [==============================] - 99s 3ms/step - loss: 0.0189
Epoch 2/10
35062/35062 [==============================] - 97s 3ms/step - loss: 0.0180
Epoch 3/10
35062/35062 [==============================] - 98s 3ms/step - loss: 0.0179
Epoch 4/10
35062/35062 [==============================] - 98s 3ms/step - loss: 0.0178
Epoch 5/10
35062/35062 [==============================] - 95s 3ms/step - loss: 0.0177
Epoch 6/10
35062/35062 [==============================] - 97s 3ms/step - loss: 0.0177
Epoch 7/10
35062/35062 [==============================] - 97s 3ms/step - loss: 0.0177
Epoch 8/10
35062/35062 [==============================] - 96s 3ms/step - loss: 0.0177
Epoch 9/10
35062/35062 [==============================] - 95s 3ms/step - loss: 0.0176
Epoch 10/10
1/1 [==============================] - 0s 23ms/step


,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%)
0,0.225908,0.051667,0.025575,0.0,0.548
1,0.218009,0.000000,0.005115,0.0,0.570
2,0.206951,0.000000,0.005115,0.0,0.631
3,0.200632,0.632222,0.015345,0.0,0.675
4,0.189573,0.000000,0.002558,0.0,0.705
...,...,...,...,...,...
7723,0.426540,0.868333,0.071611,0.0,0.411
7724,0.413902,0.840278,0.061381,0.0,0.437
7725,0.396524,0.962778,0.040921,0.0,0.425
7726,0.388626,1.000000,0.040921,0.0,0.509


In [ ]:
filled_seonggeo_aws.to_csv('/content/drive//MyDrive/TNT공모전/test_aws/seonggeo_aws.csv', index=False, encoding='utf-8-sig')

3일치 기상데이터 예측된 파일 불러와 예측 실행

In [ ]:
filename_to_var = {'asan_aws.csv': 'asan_aws_filled', 'daecheonh_aws.csv': 'daecheonh_aws_filled', 
                   'gongju_aws.csv': 'gongju_aws_filled', 'gyeryong_aws.csv': 'gyeryong_aws_filled', 
                   'hongbuk_aws.csv': 'hongbuk_aws_filled', 'nonsan_aws.csv': 'nonsan_aws_filled', 
                   'oworld_aws.csv': 'oworld_aws_filled', 'secheon_aws.csv': 'secheon_aws_filled', 
                   'seonggeo_aws.csv': 'seonggeo_aws_filled', 'sjgoun_aws.csv': 'sjgoun_aws_filled', 
                   'sjjeonui_aws.csv': 'sjjeonui_aws_filled', 'sjyeonseo_aws.csv': 'sjyeonseo_aws_filled', 
                   'taean_aws.csv': 'taean_aws_filled', 'yesan_aws.csv': 'yesan_aws_filled', 'daesan_aws.csv': 'daesan_aws_filled'}

test_folder_path = "/content/drive/MyDrive/TNT공모전/test_aws"

for filename, variable_name in filename_to_var.items():
    file_path = f"{test_folder_path}/{filename}"
    globals()[variable_name] = pd.read_csv(file_path, encoding='utf-8')

    print(f"{filename} 저장된 변수명: {variable_name}")

asan_aws.csv 저장된 변수명: asan_aws_filled
daecheonh_aws.csv 저장된 변수명: daecheonh_aws_filled
gongju_aws.csv 저장된 변수명: gongju_aws_filled
gyeryong_aws.csv 저장된 변수명: gyeryong_aws_filled
hongbuk_aws.csv 저장된 변수명: hongbuk_aws_filled
nonsan_aws.csv 저장된 변수명: nonsan_aws_filled
oworld_aws.csv 저장된 변수명: oworld_aws_filled
secheon_aws.csv 저장된 변수명: secheon_aws_filled
seonggeo_aws.csv 저장된 변수명: seonggeo_aws_filled
sjgoun_aws.csv 저장된 변수명: sjgoun_aws_filled
sjjeonui_aws.csv 저장된 변수명: sjjeonui_aws_filled
sjyeonseo_aws.csv 저장된 변수명: sjyeonseo_aws_filled
taean_aws.csv 저장된 변수명: taean_aws_filled
yesan_aws.csv 저장된 변수명: yesan_aws_filled
daesan_aws.csv 저장된 변수명: daesan_aws_filled


In [ ]:
filled_test_AWS_lst = [asan_aws_filled, daecheonh_aws_filled, gongju_aws_filled, gyeryong_aws_filled, hongbuk_aws_filled, nonsan_aws_filled, 
                       oworld_aws_filled, secheon_aws_filled, seonggeo_aws_filled, sjgoun_aws_filled, sjjeonui_aws_filled, sjyeonseo_aws_filled, 
                       taean_aws_filled, yesan_aws_filled, daesan_aws_filled]
                       
for loc in filled_test_AWS_lst:
  loc = pd.DataFrame(loc)

In [ ]:
filled_asan_aws = pd.concat([asan_aws_t.iloc[:, 1], asan_aws_filled], axis=1)
filled_daecheonh_aws = pd.concat([daecheonh_aws_t.iloc[:, 1], daecheonh_aws_filled], axis=1)
filled_gongju_aws = pd.concat([gongju_aws_t.iloc[:, 1], gongju_aws_filled], axis=1)
filled_gyeryong_aws = pd.concat([gyeryong_aws_t.iloc[:, 1], gyeryong_aws_filled], axis=1)
filled_hongbuk_aws = pd.concat([hongbuk_aws_t.iloc[:, 1], hongbuk_aws_filled], axis=1)
filled_nonsan_aws = pd.concat([nonsan_aws_t.iloc[:, 1], nonsan_aws_filled], axis=1)
filled_oworld_aws = pd.concat([oworld_aws_t.iloc[:, 1], oworld_aws_filled], axis=1)
filled_secheon_aws = pd.concat([secheon_aws_t.iloc[:, 1], secheon_aws_filled], axis=1)
filled_seonggeo_aws = pd.concat([seonggeo_aws_t.iloc[:, 1], seonggeo_aws_filled], axis=1)
filled_sjgoun_aws = pd.concat([sjgoun_aws_t.iloc[:, 1], sjgoun_aws_filled], axis=1)
filled_sjjeonui_aws = pd.concat([sjjeonui_aws_t.iloc[:, 1], sjjeonui_aws_filled], axis=1)
filled_sjyeonseo_aws = pd.concat([sjyeonseo_aws_t.iloc[:, 1], sjyeonseo_aws_filled], axis=1)
filled_taean_aws = pd.concat([taean_aws_t.iloc[:, 1], taean_aws_filled], axis=1)
filled_yesan_aws = pd.concat([yesan_aws_t.iloc[:, 1], yesan_aws_filled], axis=1)
filled_daesan_aws = pd.concat([daesan_aws_t.iloc[:, 1], daesan_aws_filled], axis=1)

In [ ]:
file_pm_to_var = {"공주.csv": "gongju_pm_t", "노은동.csv": "noeun_pm_t", "논산.csv": "nonsan_pm_t", 
                  "대천2동.csv": "daecheon2_pm_t", "독곶리.csv": "dokgot_pm_t", "동문동.csv": "dongmun_pm_t", 
                  "모종동.csv": "mojong_pm_t", "문창동.csv": "munchang_pm_t", "성성동.csv": "seongseong_pm_t", 
                  "신방동.csv": "sinbang_pm_t", "신흥동.csv": "sinheung_pm_t", "아름동.csv": "areum_pm_t", 
                  "예산군.csv": "yesan_pm_t", "읍내동.csv": "eupnae_pm_t", "이원면.csv": "iwon_pm_t", 
                  "정림동.csv": "jeonglim_pm_t", "홍성읍.csv": "hongseong_pm_t"}

test_pm_path = "/content/drive/MyDrive/TNT공모전/dataset/TEST_INPUT"

for filename, variable_name in file_pm_to_var.items():
    file_path = f"{test_pm_path}/{filename}"
    globals()[variable_name] = pd.read_csv(file_path, encoding='utf-8')

    print(f"{filename} 저장된 변수명: {variable_name}")

공주.csv 저장된 변수명: gongju_pm_t
노은동.csv 저장된 변수명: noeun_pm_t
논산.csv 저장된 변수명: nonsan_pm_t
대천2동.csv 저장된 변수명: daecheon2_pm_t
독곶리.csv 저장된 변수명: dokgot_pm_t
동문동.csv 저장된 변수명: dongmun_pm_t
모종동.csv 저장된 변수명: mojong_pm_t
문창동.csv 저장된 변수명: munchang_pm_t
성성동.csv 저장된 변수명: seongseong_pm_t
신방동.csv 저장된 변수명: sinbang_pm_t
신흥동.csv 저장된 변수명: sinheung_pm_t
아름동.csv 저장된 변수명: areum_pm_t
예산군.csv 저장된 변수명: yesan_pm_t
읍내동.csv 저장된 변수명: eupnae_pm_t
이원면.csv 저장된 변수명: iwon_pm_t
정림동.csv 저장된 변수명: jeonglim_pm_t
홍성읍.csv 저장된 변수명: hongseong_pm_t


In [ ]:
pm_test_data = [gongju_pm_t, noeun_pm_t, nonsan_pm_t, daecheon2_pm_t, dokgot_pm_t, dongmun_pm_t, mojong_pm_t, munchang_pm_t, seongseong_pm_t, 
                sinbang_pm_t, sinheung_pm_t, areum_pm_t, yesan_pm_t, eupnae_pm_t, iwon_pm_t, jeonglim_pm_t, hongseong_pm_t]

loc_pm_test = ['gongju_pm_test', 'noeun_pm_test', 'nonsan_pm_test', 'daecheon2_pm_test', 'dokgot_pm_test', 'dongmun_pm_test', 'mojong_pm_test', 
               'munchang_pm_test', 'seongseong_pm_test', 'sinbang_pm_test', 'sinheung_pm_test', 'areum_pm_test', 'yesan_pm_test', 'eupnae_pm_test', 
               'iwon_pm_test', 'jeonglim_pm_test', 'hongseong_pm_test']

for loc, name in zip(pm_test_data, loc_pm_test):
  locals()[name] = loc.drop(columns=['연도', '측정소'])

In [ ]:
test_data_aws = pd.concat([filled_gongju_aws, filled_gyeryong_aws, filled_nonsan_aws, filled_daecheonh_aws, 
                           filled_daesan_aws, filled_taean_aws, filled_asan_aws, filled_oworld_aws, 
                           filled_seonggeo_aws, filled_sjjeonui_aws, filled_sjyeonseo_aws, filled_sjgoun_aws, 
                           filled_yesan_aws, filled_secheon_aws, filled_taean_aws, filled_oworld_aws, 
                           filled_hongbuk_aws], ignore_index=True)

In [ ]:
test_data_aws

,일시,기온(°C),풍향(deg),풍속(m/s),강수량(mm),습도(%)
0,01-01 00:00,0.244866,0.123333,0.038363,0.0,0.647
1,01-01 01:00,0.232227,0.167778,0.033248,0.0,0.648
2,01-01 02:00,0.206951,0.000000,0.002558,0.0,0.734
3,01-01 03:00,0.199052,0.000000,0.002558,0.0,0.753
4,01-01 04:00,0.189573,0.000000,0.002558,0.0,0.795
...,...,...,...,...,...,...
131371,11-18 19:00,0.404423,0.833611,0.089514,0.0,0.554
131372,11-18 20:00,0.396525,0.830556,0.074169,0.0,0.552
131373,11-18 21:00,0.393365,0.837500,0.084399,0.0,0.579
131374,11-18 22:00,0.387046,0.806944,0.094629,0.0,0.593


In [ ]:
test_data_pm = pd.concat([gongju_pm_test, noeun_pm_test, nonsan_pm_test, daecheon2_pm_test, dokgot_pm_test, 
                          dongmun_pm_test, mojong_pm_test, munchang_pm_test, seongseong_pm_test, sinbang_pm_test, 
                          sinheung_pm_test, areum_pm_test, yesan_pm_test, eupnae_pm_test, iwon_pm_test, jeonglim_pm_test, hongseong_pm_test], ignore_index=True)

test_data_pm.columns = ['일시2', 'PM2.5']

In [ ]:
#test_data_aws = 각각에 대해 딥러닝으로 처리된 데이터가 담긴 변수
df_test_r = pd.concat([test_data_aws, test_data_pm], axis=1)
df_test_r.drop(columns=['일시2'], inplace=True)

df_test_r.columns = ['date', 'temp', 'wind_d', 'wind_s', 'prec', 'humid', 'PM2.5']
df_test_r


,date,temp,wind_d,wind_s,prec,humid,PM2.5
0,01-01 00:00,0.244866,0.123333,0.038363,0.0,0.647,0.060
1,01-01 01:00,0.232227,0.167778,0.033248,0.0,0.648,0.064
2,01-01 02:00,0.206951,0.000000,0.002558,0.0,0.734,0.072
3,01-01 03:00,0.199052,0.000000,0.002558,0.0,0.753,0.064
4,01-01 04:00,0.189573,0.000000,0.002558,0.0,0.795,0.056
...,...,...,...,...,...,...,...
131371,11-18 19:00,0.404423,0.833611,0.089514,0.0,0.554,0.056
131372,11-18 20:00,0.396525,0.830556,0.074169,0.0,0.552,0.036
131373,11-18 21:00,0.393365,0.837500,0.084399,0.0,0.579,0.020
131374,11-18 22:00,0.387046,0.806944,0.094629,0.0,0.593,0.024


In [ ]:
#학습된 선형회귀모델로 예측
test_data = df_test_r.drop(columns=['date'])

missing_rows = test_data[test_data['PM2.5'].isnull()]

X_missing = missing_rows.drop(["PM2.5"], axis=1)

interaction_features_new = X_missing[['temp', 'wind_d', 'prec']]
non_interaction_features_new = X_missing[['wind_s', 'humid']]

X_new_interaction = interaction.transform(interaction_features_new)
X_new_combined = np.hstack([X_new_interaction, non_interaction_features_new])

predicted_pm = model_no_outliers.predict(X_new_combined)

predicted_pm = pd.DataFrame(predicted_pm)
predicted_pm

,0
0,0.109966
1,0.109966
2,0.109966
3,0.108900
4,0.108900
...,...
78331,0.076840
78332,0.076840
78333,0.076278
78334,0.076278


In [ ]:
answer = pd.read_csv("/content/drive/MyDrive/TNT공모전/dataset/answer_sample.csv")

In [ ]:
answer_iloc = answer.iloc[:, :3]
answer_iloc.columns = ['year', 'date', 'pm_check']

In [ ]:
final = pd.concat([answer_iloc, predicted_pm], axis=1, ignore_index=True)
final.columns=['연도', '일시', '측정소', 'PM2.5']

In [ ]:
final.to_csv('/content/drive//MyDrive/TNT공모전/FINAL_RESULT_gyoho.csv', index=False, encoding='utf-8-sig')

In [ ]:
final

,연도,일시,측정소,PM2.5
0,4,01-03 00:00,공주,0.109966
1,4,01-03 01:00,공주,0.109966
2,4,01-03 02:00,공주,0.109966
3,4,01-03 03:00,공주,0.108900
4,4,01-03 04:00,공주,0.108900
...,...,...,...,...
78331,4,11-16 19:00,홍성읍,0.076840
78332,4,11-16 20:00,홍성읍,0.076840
78333,4,11-16 21:00,홍성읍,0.076278
78334,4,11-16 22:00,홍성읍,0.076278
